In [1]:
flow_len = 0.5


import pandas as pd

df = pd.read_csv('friends.csv')
df['raw_text'] = df['raw_text'].apply(lambda x: [[i.split('</b>:')[0].replace('<b>', ''), i.split('</b>:')[1]] for i in x.split('<br>') if "</b>:" in i ])
df['raw_text'] = df['raw_text'].apply(lambda x: x[:int(len(x)*flow_len)])

In [2]:
def process(r):
    text_list = r['raw_text']
    speaker = r['characters']
    ans = ''
    for text in text_list:
        if text[0] == speaker:
            ans += 'Speaker' + ' : ' + text[1] + ', '
        else:
            ans += 'Others' + ' : ' + text[1] + ', '
    ans += '; '
    return ans

def process_only_speaker(row):
    text_list = row['raw_text']
    speaker = row['characters']
    
    ans = ''
    for text in text_list:
        if text[0] == speaker:
            ans +=  text[1] + ', '
    ans += '; '
    return ans

df['origin_sent'] = df.apply(process, axis=1)
df['only_speaker'] = df.apply(process_only_speaker, axis=1)

df.head()

,Unnamed: 0,characters,raw_text,scene_id,cAGR,cCON,cEXT,cOPN,cNEU,single_text,full,single_context,origin_sent,only_speaker
0,0,Phoebe Buffay,"[[Monica Geller, Well, what happened?], [Phoe...",01_e12_c08(0),False,False,True,True,True,"speaker0 Well, he came in for a massage, and ...","speaker1 Well, what happened? speaker0 Well, ...","speaker0 Well, he came in for a massage, and ...","Others : Well, what happened?, Speaker : Wel...","Well, he came in for a massage, and everythin..."
1,1,Rachel,"[[Monica, (to Chandler) Anything but stew.], ...",03_e24_c04(0),False,False,True,True,False,"speaker0 Yeah. speaker0 Oh, ah with who?",speaker1 (to Chandler) Anything but stew. spe...,"speaker0 Yeah. speaker0 Oh, ah with who? spea...","Others : (to Chandler) Anything but stew., Ot...",;
2,2,Eddie,"[[Eddie, Hey pal.], [Chandler Bing, Ahhhh-ga...",02_e19_c06(0),True,True,True,True,True,speaker0 Hey pal. speaker1 Ahhhh-gaaaahhh. Ed...,speaker0 Hey pal. speaker1 Ahhhh-gaaaahhh. Ed...,speaker0 Hey pal. speaker1 Ahhhh-gaaaahhh. Ed...,"Speaker : Hey pal., Others : Ahhhh-gaaaahhh....","Hey pal., Ah, just some basic dehydrating of..."
3,3,Phoebe Buffay,"[[Chandler Bing, Oh no no no, she's a total w...",02_e12_c05(0),True,False,True,True,True,"speaker0 No, uh-uh, I'm just, I'm nervous. So...","speaker1 Oh no no no, she's a total wack job....","speaker0 No, uh-uh, I'm just, I'm nervous. So...","Others : Oh no no no, she's a total wack job....",;
4,4,Chandler Bing,"[[Joey Tribbiani, Uh, hey, Dr. Greene, why do...",02_e22_c03(1),False,False,False,False,False,"speaker0 Yes because uh, you look so young. s...","speaker1 Uh, hey, Dr. Greene, why don't you c...","speaker0 Yes because uh, you look so young. s...","Others : Uh, hey, Dr. Greene, why don't you c...","Yes because uh, you look so young., ;"


# add emotions

In [3]:
df_with_emo = pd.read_csv('../data/Friends_A_vad.tsv', sep='\t')
print(df_with_emo.shape)
df['Dialog_EmoBERTa_label'] =  df_with_emo['Dialog_EmoBERTa_label']

(711, 15)


In [4]:
def get_emotion_prompt(r):
    dialog_state = r['dialog_state']
    emotions = eval(r['Dialog_EmoBERTa_label'])
    character = r['characters'].split()[0]
    ans = ''
    for i in range(len(dialog_state)):
        if i == 0:
            if dialog_state[i] == 0:
                ans += 'First, the emotion of others is ' + emotions[i] + ', '
            elif dialog_state[i] == 1:
                ans += 'The emotion of ' + character + ' is initially ' + emotions[i] + ', '
        else:
            if dialog_state[i] == 0:
                ans += 'the emotion of others is ' + emotions[i] + ', '
            elif dialog_state[i] == 1:
                ans += character + ' respond with ' + emotions[i] + ', '
    return ans

def get_seg_id(sent_list, role):
    '''
    Generate the segment id for the whole sent
    '''

    ans = []
   
    for i in sent_list:
        if i[0].split(' ')[0] != role.split(' ')[0]:
            
            ans.append(0)
        else:
            ans.append(1)
    return ans

df['dialog_state'] = df.apply(lambda r: get_seg_id(r['raw_text'], r['characters']), axis=1)
df['affective_prompt'] = df.apply(get_emotion_prompt, axis=1)
df['sent_and_prompt'] = df['origin_sent'] + df['affective_prompt']

In [5]:
df_A = df[['scene_id', 'raw_text', 'characters', 'affective_prompt', 'only_speaker', 'origin_sent', 'sent_and_prompt', 'cAGR']]
df_C = df[['scene_id', 'raw_text', 'characters', 'affective_prompt', 'only_speaker', 'origin_sent', 'sent_and_prompt', 'cCON']]
df_E = df[['scene_id', 'raw_text', 'characters', 'affective_prompt', 'only_speaker', 'origin_sent', 'sent_and_prompt', 'cEXT']]
df_O = df[['scene_id', 'raw_text', 'characters', 'affective_prompt', 'only_speaker', 'origin_sent', 'sent_and_prompt', 'cOPN']]
df_N = df[['scene_id', 'raw_text', 'characters', 'affective_prompt', 'only_speaker', 'origin_sent', 'sent_and_prompt', 'cNEU']]

df_A['labels'] = df_A['cAGR'].apply(lambda x: 1 if x is True else 0)
df_C['labels'] = df_C['cCON'].apply(lambda x: 1 if x is True else 0)
df_E['labels'] = df_E['cEXT'].apply(lambda x: 1 if x is True else 0)
df_O['labels'] = df_O['cOPN'].apply(lambda x: 1 if x is True else 0)
df_N['labels'] = df_N['cNEU'].apply(lambda x: 1 if x is True else 0)

df_A.to_csv('Friends_A_with_role.tsv', sep='\t') 
df_C.to_csv('Friends_C_with_role.tsv', sep='\t')
df_E.to_csv('Friends_E_with_role.tsv', sep='\t')
df_O.to_csv('Friends_O_with_role.tsv', sep='\t')
df_N.to_csv('Friends_N_with_role.tsv', sep='\t')

/tmp/ipykernel_3477812/4279438878.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_A['labels'] = df_A['cAGR'].apply(lambda x: 1 if x is True else 0)
/tmp/ipykernel_3477812/4279438878.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_C['labels'] = df_C['cCON'].apply(lambda x: 1 if x is True else 0)
/tmp/ipykernel_3477812/4279438878.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cavea

## Personality Description

In [6]:
personality_description = {
    'A_pos': 'is friendly, cooperative, empathetic, and compassionate, often prioritizing harmonious relationships and the well-being of others.',
    'A_neg': 'is confrontational, uncooperative, lacking empathy, and often prioritizing their own needs and desires over the well-being of others.',
    'C_pos': 'is organized, responsible, diligent, detail-oriented, and committed to achieving their goals with a strong sense of duty and self-discipline.',
    'C_neg': 'is disorganized, careless, impulsive, lacking discipline, and often displaying a disregard for responsibilities and commitments.',
    'E_pos': 'is outgoing, sociable, energetic, and thriving in social interactions, often seeking stimulation and enjoying the company of others.',
    'E_neg': 'is introverted, reserved, quiet, and often preferring solitude or smaller social settings, conserving energy and finding fulfillment in introspection and reflection.',
    'O_pos': 'has curiosity, open-mindedness, creativity, tolerance, emotional expressiveness, and willingness to embrace new experiences and ideas.',
    'O_neg': 'is closed-minded, resistant to change, lacking curiosity, intolerant of differences, emotionally guarded, and hesitant to explore new ideas or experiences.',
    'N_pos': 'is prone to experiencing negative emotions, such as anxiety, worry, and mood swings, often displaying heightened sensitivity to stress and a tendency towards self-doubt and emotional instability.',
    'N_neg': 'is emotionally stable, resilient, and composed, often displaying a calm and balanced demeanor, and having a tendency to handle stress and adversity with ease.'
}

def get_personailty_description(r, p):
    character = r['characters'].split()[0]
    label = r['labels']
    key = p+ '_pos'
    # if label == 1:
    #     key = p+ '_pos'
    # else:
    #     key = p+ '_neg'
    return character + ' ' + personality_description[key]

personality = ['A', 'C', 'E', 'O', 'N']
for p in personality:
    print(p, '...')
    df_tmp = pd.read_csv('Friends_'+p+'_with_role.tsv', sep='\t')
    df_tmp['personality_description'] = df_tmp.apply(get_personailty_description, p=p, axis=1)
    
    
    df_tmp['sent'] = df_tmp['origin_sent']
#     df_tmp['sent'] = df_tmp['only_speaker']
    df_tmp['sentence1'] = df_tmp['sent_and_prompt']
    df_tmp['sentence2'] = df_tmp['personality_description']
    
    df_tmp.to_csv('Friends_'+p+'_with_role_'+str(flow_len)+'.tsv', sep='\t', index=False)


A ...
C ...
E ...
O ...
N ...


In [7]:
df_labels = df[['cAGR','cCON','cEXT','cOPN','cNEU']]
df_counts = pd.DataFrame([])
df_counts['AGR'] = df_labels['cAGR'].value_counts().reset_index()['cAGR']
df_counts['CON'] = df_labels['cCON'].value_counts().reset_index()['cCON']
df_counts['EXT'] = df_labels['cEXT'].value_counts().reset_index()['cEXT']
df_counts['OPN'] = df_labels['cOPN'].value_counts().reset_index()['cOPN']
df_counts['NEU'] = df_labels['cNEU'].value_counts().reset_index()['cNEU']
df_counts

,AGR,CON,EXT,OPN,NEU
0,405,381,399,462,379
1,306,330,312,249,332
